In [39]:
%pip install requests ddgs pandas

Note: you may need to restart the kernel to use updated packages.


In [40]:
from ddgs import DDGS 
import requests
import os
import pandas as pd

In [41]:
save_folder = "downloaded_images"
os.makedirs(save_folder, exist_ok=True)

def download_images(query, max_results=5):
    success = 0
    with DDGS() as ddgs:
        results = ddgs.images(
            query=query, 
            max_results=max_results
        )

        for i, img in enumerate(results):
            url = img['image']
            try:
                response = requests.get(url, timeout=10)
                response.raise_for_status()
                
                filename = f'{save_folder}/{query.replace(" ", "_")}_{i+1}.jpg'

                with open(filename, 'wb') as f:
                    f.write(response.content)
                print(f'Downloaded: {filename}')
                success += 1
            except requests.exceptions.RequestException as e:
                print(f'Failed to download {url}: {e}')

    print(f'Total images downloaded for "{query}": {success}/{max_results}')

# download_images("Hanging Bridge in Hagonoy", max_results=5)

Load the CSV file into a DataFrame

- Take only the 'name' and 'municipality' columns
- Turn it into a list of tuples
- Make a function that will return this: f{name} in {municipality}


In [42]:
df = pd.read_csv('poi.csv')
df = df[['name', 'municipality']]
df_dict = df.to_dict(orient='records')
df_dict

def generate_search_queries(poi_list):
    queries = []
    for poi in poi_list:
        name = poi['name']
        municipality = poi['municipality']
        query = f"{name} in {municipality} Bulacan"
        queries.append(query)
    return queries

search_queries = generate_search_queries(df_dict)